## 인증(Authentication) - Passport 

-  passport, passport-local, express-session 등을 이용하여 **사용자 등록 및 인증**이 가능한 사이트를 구축해 봅니다.

<img src="./login-0.png" > <div align="center"><b>Login page</b></div>
<img src="./register-0.png"> <div align="center"><b>Register page</b></div>
<img src="./index-0.png"> <div align="center"><b>index page</b></div>


- 개발의 편의를 위해서 사용자 정보는 DB 대신에 프로그램 내의 배열로 저장해서 사용합니다.


- 다음과 같은 일련의 명령을 사용하여 폴더를 만들고 npm 환경 구축을 시작합니다.
```
$ mkdir server
$ cd server
$ npm init -y
```

- 그런 다음 dependencies 들을 설치하십시오.
```
$ npm i express pug passport passport-local bcrypt express-flash express-session mongoose
```
  - passport : 인증
  - passport-local : local 컴퓨터에서의 인증
  - bcrypt : 암호 인코딩 (hashed password)
  - express-flash : 오류발생시 메시지 개시
  - express-session : 세션기능 
  
  

- 생성할 앱의 MVC 구조를 반영한 디렉토리 구조는 다음과 같습니다.

```
├── app.js
├── package.json
├── models (not used in this example)
├── public (not used in this example)
├── routes
│ ├── login.js
│ └── passport-config.js
└── views
│ ├── index.pug
│ ├── layout.pug
│ ├── login.pug
│ └── register.pug

```

* (source code)  /app.js

In [ ]:
const express = require('express')
const router = require('./routes/login')
const flash = require('express-flash')
const session = require('express-session')
const passport = require('passport')

const app = express()

// Views
app.set('views','./views')
app.set('view engine', 'pug')

// Middlewares
app.use(express.urlencoded({ extended: false}))
app.use(flash())
app.use(session({
    secret: 'secret',
    resave: false,
    saveUninitialized: true
}))
app.use(passport.initialize())
app.use(passport.session())

// Routers
app.use('/', router)

// Start server
app.listen(3000, console.log('Server started at 3000'))

* /routes/login.js

In [ ]:
const express = require('express')
const passport = require('passport')
const bcrypt = require('bcrypt')
const router = express.Router()

const users = []

const initializePassport = require('./passport-config')

initializePassport(
    passport, 
    email => users.find(user => user.email === email),
    id => users.find(user => user.id === id)
)

router.get('/', ensureAuthenticated, (req, res) => {
    res.render('index', {name: req.user.name})
})

router.get('/login', forwardAuthenticated, (req, res) => {
    res.render('login')
})

router.post('/login', forwardAuthenticated, passport.authenticate('local', {
    successRedirect: '/',
    failureRedirect: '/login',
    failureFlash: true
}))

router.get('/register', forwardAuthenticated, (req, res) => {
    res.render('register')
})

router.post('/register', forwardAuthenticated, (req, res) => {
    bcrypt.hash(req.body.password, 10)
        .then(hashedPassword => {
            users.push({
                id: Date.now().toString(),
                name: req.body.name,
                email: req.body.email,
                password: hashedPassword
            })
            res.redirect('/login')
            console.log(users)
        })
        .catch(err => {
            console.log(err)
            res.redirect('/register')
        })
})

router.post('/logout', (req, res) => {
    req.logout()
    res.redirect('/login')
})

function ensureAuthenticated(req, res, next) {
    if(req.isAuthenticated()) {
        return next()
    } 
    res.redirect('/login')
}

function forwardAuthenticated(req, res, next) {
    if(req.isAuthenticated()) {
        res.redirect('/')
    } 
    next()
}

module.exports = router

* /routes/passport-config.js

In [ ]:
const passport = require('passport')
const bcrypt = require('bcrypt')
const LocalStrategy = require('passport-local').Strategy

function initialize(passport, getUserByEmail, getUserById) {
    
    const authenticateUser = (email, password, done) => {
        const user = getUserByEmail(email)
        if (user == null) {
            return done(null, false, {message: 'No user with that email'})
        }
        bcrypt.compare(password, user.password)
            .then(same => {
                if(same) { return done(null, user) }
                else { return done(null, false, { message: 'Password incorrect'})}
                })
            .catch(err => { console.log(err) }) 
    }
    
    passport.use(new LocalStrategy({usernameField: 'email'}, authenticateUser))
    passport.serializeUser((user, done) => { done(null, user.id) })
    passport.deserializeUser((id, done) => {
        return done(null, getUserById(id))
    })
};

module.exports = initialize

* /views/index.pug

In [ ]:
doctype html
html
    head
    body
        h1 Hello 
            =name
        hr
        form(action='/logout' method='POST') 
            button(type='submit') Logout

* /views/login.pug

In [ ]:
h1 Login
if(messages.error)
    p= messages.error
form(action='/login' method='POST')
    label(for='email') Email
    input(type='email' id='email' name='email' required)
    br
    label(for='password') Password
    input(type='password' id='password' name='password' required)
    br
    button(type='submit') Login
    hr
    a(href='/register') Register

* /views/register.pug

In [ ]:
extends ./layout.pug
block content
    div(class='container')
        div(class="row mt-4")
            div(class="col-md-3 m-auto")
                div(class="card card-body text-center")
                    h1(class="mb-3") Register
                    form(action='/register' method='POST')
                        label(for='name') Name : 
                        input(type='text' id='name' name='name' required)
                        br
                        label(for='email') Email : 
                        input(type='email' id='email' name='email' required)
                        br
                        label(for='password') Password :
                        input(type='password' id='password' name='password' required)
                        br
                        br
                        button(type='submit') Register
                        hr
                        a(href='/login') 
                            b Login

* /views/layout.pug

In [ ]:
doctype html
html
    head
        title= title
        meta(meta charset="utf-8")
        meta(name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no")
        link(rel='stylesheet', href='https://bootswatch.com/4/journal/bootstrap.min.css')
    body
        block content

### 코드 개선하기
* login.pug 를 수정하여, http://localhost/register 페이지처럼 보이도록 수정하기 (참고 링크: [bootswatch.com](https://bootswatch.com/journal/) )

* 

## 사용자 등록 및 인증 (DB 활용) 

* 이번 예제에서는 사용자 정보를 mongoDB 에 저장하고 이를 활용한 인증하는 응용 웹서버를 좀 더 완성도가 높도록 Bootstrap 와 awesome 폰트를 활용하여 다음과 같이 작성해 봅니다.

<img src="index2.png" width="300"> <div align='center'> <b> Front page </b> </div>

<img src="login.png" width="300"> <div align='center'> <b> Login page </b> </div>

<img src="register.png" width="300"> <div align='center'> <b> Register page </b> </div>

<img src="dashboard.png" width="300"> <div align='center'> <b> Dashboard page </b> </div>

* 다음과 같은 일련의 명령을 사용하여 폴더를 만들고 npm 환경 구축을 시작합니다.
```
$ mkdir server2
$ cd server2
$ npm init -y
```

* dependencies 들은 다음과 같이 설치하십시오.

$ npm i express pug passport passport-local bcrypt express-flash express-session mongoose

  - passport : 인증
  - passport-local : local 컴퓨터에서의 인증
  - bcrypt : 암호 인코딩
  - express-flash : 오류발생시 메시지 개시
  - express-session : 세션기능
  - mongoose : mongodb 연결

* 생성할 앱의 MVC 구조를 반영한 디렉토리 구조는 다음과 같습니다.

```
├── app.js
├── package.json
├── passport-config.js
├── models
│ └── user.js
├── public
├── routes
│ ├── auth.js
│ ├── index.js
│ └── users.js
└── views
│ ├── dashboard.pug
│ ├── index.pug
│ ├── layout.pug
│ ├── login.pug
│ └── register.pug
```

* app.js

In [ ]:
const express = require('express')
const flash = require('express-flash')
const session = require('express-session')
const passport = require('passport')

const index = require('./routes/index')
const users = require('./routes/users')

const app = express()

// Passport config
const initializePassport = require('./passport-config')
initializePassport(passport)

// Views in pug
app.set('views', './views')
app.set('view engine', 'pug')

// BodyParser
app.use(express.urlencoded({ extended: false}))

// Express Session
app.use(session({
    secret: 'secret',
    resave: false,
    saveUninitialized: true
}))

// Passport middleware
app.use(passport.initialize())
app.use(passport.session())

// Express flash
app.use(flash())

// Global variables
app.use((req, res, next) => {
    res.locals.success_msg = req.flash('success_msg'),
    res.locals.error_msg = req.flash('error_msg'),
    res.locals.error = req.flash('error'),
    next()
})

// Routes
app.use('/', index)
app.use('/users', users)

app.listen(3000, console.log(`Start server on port 3000`))

* passport-config.js

In [ ]:
const LocalStrategy = require('passport-local').Strategy
const mongoose = require('mongoose')
const User = require('./models/user')
const bcrypt = require('bcrypt')

function initialize(passport) {
    const authenticateUser = (email, password, done) => {
        User.findOne({ email: email })
            .then(user => {
                if(!user) {
                    return done(null, false, { message: 'That email is not registered'})
                }
                bcrypt.compare(password, user.password)
                    .then(same => {
                        if(same) { return done(null, user) }
                        else { return done(null, false, { message: 'Password incorrect'})}
                        })
                    .catch(err => { console.log(err) }) 
            })
            .catch(err => console.log('err'))
    }
    
    passport.use(new LocalStrategy({ usernameField: 'email'}, authenticateUser))
    passport.serializeUser((user, done) => { done(null, user.id) })
    passport.deserializeUser((id, done) => {
        User.findById(id, (err, user) => { done(null, user.id) })
    })
}

module.exports = initialize

* /models/user.js

In [ ]:
const mongoose = require('mongoose')

const UserSchema = new mongoose.Schema({
    name: {
        type: String,
        required: true
    },
    email: {
        type: String,
        required: true
    },
    password: {
        type: String,
        required: true
    },
    date: {
        type: Date,
        default: Date.now
    }
})

module.exports = mongoose.model('User', UserSchema)

* /routes/index.js

In [ ]:
const express = require('express')
const router = express.Router()
const { ensureAuthenticated, forwardAuthenticated } = require('./auth')

router.get('/', forwardAuthenticated, (req, res) => {
    res.render('index')
})

router.get('/dashboard', ensureAuthenticated, (req, res) => {
    res.render('dashboard', {
        name: req.user.name
    })
})

module.exports = router

* /routes/users.js

In [ ]:
const express = require('express')
const router = express.Router()
const bcrypt = require('bcrypt')
const mongoose = require('mongoose')
const passport = require('passport')
const { forwardAuthenticated } = require('./auth')

// Use Models
const User = require('../models/user') 
const db = "mongodb://127.0.0.1/usersDB"
mongoose.connect(db, { useNewUrlParser: true, useUnifiedTopology: true})


// Login page
router.get('/login', forwardAuthenticated, (req, res) => {
    res.render('login')
})

// Login handle
router.post('/login', (req, res, next) => {
    passport.authenticate('local', {
        successRedirect: '/dashboard',
        failureRedirect: '/users/login',
        failureFlash: true
    })(req, res, next)
})

// Register page
router.get('/register', forwardAuthenticated, (req, res) => {
    res.render('register')
})

// Router page in post
router.post('/register', (req, res) => {
    const { name, email, password, password2 } = req.body
    let errors = []
    // Check passwordd match
    if (password !== password2) {
        errors.push({ msg: 'Passwords do not match' })
    }
    // Check password's length
    if (password.length < 4) {
        errors.push({ msg: 'Password should be at least 4 characters' })
    }
    // Error messages
    if (errors.length > 0) {
        res.render('register', {
            errors, 
            name, email, password, password2
        })
    } else {
        // Validation passed
        User.findOne({ email: email})
            .then(user => {
                if(user) {
                    //User exists
                    errors.push({ msg: 'Email is already registered'})
                    res.render('register', {
                        errors,
                        name, email, password, password2
                    })
                } else {
                    const newUser = new User({
                        name, email, password
                    })
                    // Hash Password
                    bcrypt.hash(newUser.password, 10, (err,hash) => {
                        if(err) throw err;
                        newUser.password = hash
                        newUser.save()
                            .then(user => {
                                req.flash('success_msg', 'Your are now registered and can log in ')
                                res.redirect('/users/login')
                            })
                            .catch(err => console.log(err))
                    })         
        }
    })
}})

router.get('/logout', (req, res) => {
    req.logout()
    req.flash('success_msg', 'You are logged out')
    res.redirect('/users/login')
})

module.exports = router

* /routes/auth.js

In [ ]:
function ensureAuthenticated(req, res, next) {
    if(req.isAuthenticated()) {
        return next()
    }
    req.flash('error_msg', 'Please login in to view this resource')
    res.redirect('/users/login')
}

function forwardAuthenticated(req, res, next) {
    if(req.isAuthenticated()) {
        res.redirect('/dashboard')
    }
    return next()
}

module.exports = { ensureAuthenticated, forwardAuthenticated }

* /views/layout.pug

In [ ]:
doctype html
html
    head
        title= title
        meta(meta charset="utf-8")
        meta(name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no")
        link(rel='stylesheet' href='https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.14.0/css/all.css')
        link(rel='stylesheet', href='https://bootswatch.com/4/journal/bootstrap.min.css')
    body
        block content
        script(src="https://code.jquery.com/jquery-3.3.1.slim.min.js" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" crossorigin="anonymous")
        script(src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.6/umd/popper.min.js" integrity="sha384-wHAiFfRlMFy6i5SRaxvfOCifBUQy1xHdJ/yoi7FRNXMRBu5WHdZYu1hA6ZOblgut" crossorigin="anonymous")
        script(src="https://stackpath.bootstrapcdn.com/bootstrap/4.2.1/js/bootstrap.min.js" integrity="sha384-B0UglyR+jN6CkvvICOB2joaf5I4l3gm9GU6Hc1og6Ls7i6U/mkkaduKaBhlAXv9k" crossorigin="anonymous")


* /views/index.pug

In [ ]:
extends ./layout.pug
block content
    div(class='container')
        div(class="row mt-5")
            div(class="col-md-6 m-auto")
                div(class="card card-body text-center")
                    h1
                        i(class="fab fa-node-js fa-3x")
                    p Create an account or login
                    a(href="/users/register" class="btn btn-primary btn-block mb-2") Register
                    a(href="/users/login" class="btn btn-secondary btn-block") Login

* /views/dashboard.pug

In [ ]:
extends ./layout.pug
block content
    div(class="container")
        h1(class="mt-4") Dashboard
        p(class="lead mb-3") Welcome 
            if (name)
                =name
        hr
        a(href="/users/logout" class="btn btn-secondary") Logout

* /views/login.pug

In [ ]:
extends ./layout.pug
block content
    div(class='container')
        div(class="row mt-5")
            div(class="col-md-6 m-auto")
                div(class="card card-body")
                    h1(class="text-center mb-3")
                        i(class="fas fa-sign-in-alt")  Login
                    if errors
                        each error in errors
                            div(class="alert alert-warning alert-dismissible fade show" role="alert")= error.msg
                                button(type="button" class="close" data-dismiss="alert" aria-label="Close")
                                    span(aria-hidden="true") &times;
                    if success_msg != ''
                        div(class="alert alert-success alert-dismissible fade show" role="alert")= success_msg
                                button(type="button" class="close" data-dismiss="alert" aria-label="Close")
                                    span(aria-hidden="true") &times;
                    if error != ''
                        div(class="alert alert-warning alert-dismissible fade show" role="alert")= error
                                button(type="button" class="close" data-dismiss="alert" aria-label="Close")
                                    span(aria-hidden="true") &times;
                    form(action="/users/login" method="POST")
                        div(class="form-group")
                            label(for="email") Email
                            input(type="email" id="email" name="email" class="form-control" placeholder="Enter Email" value=email)
                        div(class="form-group")
                            label(for="password") Password
                            input(type="password" id="password" name="password" class="form-control" placeholder="Enter Password" value=password)
                        button(type='submit' class="btn btn-primary btn-block") Login
                    p(class="lead mt-4") No Account? 
                        a(href='/users/register') Register


* /views/register.pug

In [ ]:
extends ./layout.pug
block content
    div(class='container')
        div(class="row mt-5")
            div(class="col-md-6 m-auto")
                div(class="card card-body")
                    h1(class="text-center mb-3")
                        i(class="fas fa-user-plus")  Register
                    if errors
                        each error in errors
                            div(class="alert alert-warning alert-dismissible fade show" role="alert")= error.msg
                                button(type="button" class="close" data-dismiss="alert" aria-label="Close")
                                    span(aria-hidden="true") &times;
                    form(action="/users/register" method="POST")
                        div(class="form-group")
                            label(for="name") Name
                            input(type="name" id="name" name="name" class="form-control" placeholder="Enter Name" required value=name)
                        div(class="form-group")
                            label(for="email") Email
                            input(type="email" id="email" name="email" class="form-control" placeholder="Enter Email" required value=email)
                        div(class="form-group")
                            label(for="password") Password
                            input(type="password" id="password" name="password" class="form-control" placeholder="Enter Password" required value=password)
                        div(class="form-group")
                            label(for="password2") Confirm Password
                            input(type="password" id="password2" name="password2" class="form-control" placeholder="Confirm Password" required value=password2)
                        button(type='submit' class="btn btn-primary btn-block") Register
                    p(class="lead mt-4") Have An Account? 
                        a(href='/users/login') Login
